# Matrix-matrix multiplication by columns

We now look at how the FLAMEPy API can be used to implement different matrix-matrix multiplication algorithms.  

First, we create some matrices.

In [1]:
import numpy as np

m = 4
n = 3
k = 5

C = np.matrix( np.random.random( (m, n) ) )
print( 'C = ' )
print( C )

Cold = np.matrix( np.zeros( (m,n ) ) )
Cold = np.matrix( np.copy( C ) )           # an alternative way of doing a "hard" copy, in this case of a matrix
    
A = np.matrix( np.random.random( (m, k) ) )
print( 'A = ' )
print( A )

B = np.matrix( np.random.random( (k, n) ) )
print( 'B = ' )
print( B )

C = 
[[0.3055962  0.42938989 0.93214838]
 [0.46035809 0.29045494 0.7267029 ]
 [0.47122607 0.10792847 0.19259859]
 [0.32215902 0.47102025 0.59105158]]
A = 
[[0.19974052 0.44938482 0.92992425 0.40882762 0.55732111]
 [0.53425394 0.61926165 0.62445809 0.75342114 0.11925777]
 [0.13689054 0.76942975 0.6402926  0.35644277 0.20895158]
 [0.50558215 0.9008212  0.65603818 0.66937046 0.8076336 ]]
B = 
[[0.11256486 0.01243262 0.69359281]
 [0.08989721 0.51929951 0.48648917]
 [0.70117445 0.96923139 0.27405503]
 [0.51251243 0.61574154 0.11636613]
 [0.86842751 0.24505553 0.19798669]]


## <h2>The algorithm  </h2>  <image src="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/Gemm_nn_unb_var1.png" alt="Matrix-matrix multiplication by columns picture" width="80%">

<h2> The routine <code> Gemm_nn_unb_var1( A, B, C ) </code> </h2>

This routine computes $ C := A B + C $ by columns.  The "\_nn\_" means that this is the "No transpose, No transpose" case of matrix multiplication.  
The reason for this is that the operations $ C := A^T B + C $ ("\_tn\_" or "Transpose, No transpose"), $ C := A B^T + C $ ("\_nt\_" or "No transpose, Transpose"), and $ C := A^T B^T + C $ ("\_tt\_" or "Transpose, Transpose") are also encountered.  
    
The specific laff function we will use is
<ul>
<li> <code> laff.gemv( trans, alpha, A, x, beta, y ) </code> which computes 
$ y := \alpha A x + \beta y $ or $ y := \alpha A^T x + \beta y $, depending on 
        parameter <code> trans</code>.  In particular, 
        <ul>
        <li>
        <code> laff.gemv( 'No transpose', alpha, A, x, beta, y ) </code> computes $ y := \alpha A x + \beta y $.
            </li>
        <li>
        <code> laff.gemv( 'Transpose', alpha, A, x, beta, y ) </code> computes $ y := \alpha A^T x + \beta y $.
            </li>
            </ul>
    </li>
</ul>

Use the <a href="https://studio.edx.org/c4x/UTAustinX/UT.5.01x/asset/index.html"> Spark webpage</a> to generate a code skeleton.  (Make sure you adjust the name of the routine.)

In [2]:
import flame
import laff as laff

def Gemm_nn_unb_var1(A, B, C):

    BL, BR = flame.part_1x2(B, \
                            0, 'LEFT')

    CL, CR = flame.part_1x2(C, \
                            0, 'LEFT')

    while BL.shape[1] < B.shape[1]:

        B0, b1, B2 = flame.repart_1x2_to_1x3(BL, BR, \
                                             1, 'RIGHT')

        C0, c1, C2 = flame.repart_1x2_to_1x3(CL, CR, \
                                             1, 'RIGHT')

        #------------------------------------------------------------#

        laff.gemv( 'No transpose', 1.0, A, b1, 1.0, c1 )

        #------------------------------------------------------------#

        BL, BR = flame.cont_with_1x3_to_1x2(B0, b1, B2, \
                                            'LEFT')

        CL, CR = flame.cont_with_1x3_to_1x2(C0, c1, C2, \
                                            'LEFT')

    flame.merge_1x2(CL, CR, C)



In [3]:
C = np.matrix( np.copy( Cold ) )               # restore C 

Gemm_nn_unb_var1( A, B, C )

print( 'C - ( Cold + A * B )' )
print( C - ( Cold + A * B ) )

C - ( Cold + A * B )
[[ 0.00000000e+00 -4.44089210e-16  0.00000000e+00]
 [ 0.00000000e+00  2.22044605e-16  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  1.11022302e-16]
 [ 0.00000000e+00 -4.44089210e-16  0.00000000e+00]]


Bingo! It works!

## Watch the algorithm at work!

Copy and paste the code into <a href="http://edx-org-utaustinx.s3.amazonaws.com/UT501x/PictureFlame/PictureFLAME.html"> PictureFLAME </a>, a webpage where you can watch your routine in action.  Just cut and paste into the box.  

Disclaimer: we implemented a VERY simple interpreter.  If you do something wrong, we cannot guarantee the results.  But if you do it right, you are in for a treat.

If you want to reset the problem, just click in the box into which you pasted the code and hit "next" again.